Setup code

In [162]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [163]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogeneity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

In [164]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

message = query5

prompt_template = PromptTemplate.from_template(
    """
You are the AI that separates instruction and context from the query.
In this query, separate the instruction and context. Instruction and context must not have duplicate contents.
Only use given query itself, do not generate or change any words.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided, then print context to space.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": message})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

카이스트 전산학부 건물인 신뢰관에 대한 정보를 제공해주세요.
신뢰관은 카이스트 전산학부의 건물 중 하나입니다. 신뢰관의 구체적인 정보나 특징, 역사 등에 대해 알고 싶으신 경우, 신뢰관과 관련된 내용을 알려주시면 해당 정보를 제공해드릴 수 있습니다.


In [165]:
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

scontext = str(tavily.search(query=instr))

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", divide instruction into few steps.
Do not answer or calculate for each steps, and only divide instructions to the smaller problem.
Only print 1., 2., 3., ... steps to do. Each step should be a one-liner.
Refer to the context if you need : {context}
Searched context to help dividing instructions : {scontext}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context, "scontext": scontext}).split("\n")
print(answer)
print(len(answer))

['1. "카이스트 전산학부 건물인 신뢰관"을 검색합니다.', '2. 검색 결과에서 신뢰관의 구체적인 정보, 특징, 역사 등을 확인합니다.', '3. 검색 결과에서 신뢰관의 관련 이미지나 링크를 찾습니다.', '4. 신뢰관의 구체적인 정보가 검색 결과에서 나오지 않는다면, "카이스트 전산학부 신뢰관"을 검색합니다.', '5. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관"을 검색합니다.', '6. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 건물 신뢰관"을 검색합니다.', '7. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관"을 검색합니다.', '8. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관 역사"를 검색합니다.', '9. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관 특징"을 검색합니다.', '10. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관 크기"를 검색합니다.', '11. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관 구조"를 검색합니다.', '12. 검색 결과에서 신뢰관의 구체적인 정보가 나오지 않는다면, "KAIST 전산학부 신뢰관 역사"를 검색합니다.']
12


In [166]:
prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", determine instruction can be done only using context.
Just answer it in "Yes" or "No".
---
Context : {context}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

add = ""

if(answer == "No"): # find context from tavily
    context = scontext
    add = "\nAnswer with references if it exists."

네.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0 and len(context) != 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(message + add + "\nIf answer is not context-based, then only print 'I don't know about that.'")
)
answer = chain_result = chain.invoke({"history": history})

print(answer)
